# 1. 📝 Description of iMedic Agent (Google ADK)
iMedic Agent is an advanced AI-powered healthcare assistant built using Google AI Agent Development Kit (ADK). It is designed to streamline and automate key patient-care service workflows with high accuracy, fast responses, and seamless integration with medical systems.

The agent intelligently supports multiple healthcare operations, making it a unified digital companion for both patients and clinics.

# 2. 🛠 Setup Google API key

In [1]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

✅ Gemini API key setup complete.


# 3. 🔨 Build your agent with ADK
Set up Environment & Install ADK

In [2]:
from google.adk.agents import Agent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search, AgentTool, ToolContext
from google.genai import types
from google.adk.agents import LlmAgent
from google.adk.sessions import InMemorySessionService

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [3]:
import uuid
from google.genai import types

from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService

from google.adk.tools.mcp_tool.mcp_toolset import McpToolset
from google.adk.tools.tool_context import ToolContext
from google.adk.tools.mcp_tool.mcp_session_manager import StdioConnectionParams
from mcp import StdioServerParameters

from google.adk.apps.app import App, ResumabilityConfig
from google.adk.tools.function_tool import FunctionTool

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [4]:
# Define helper functions that will be reused throughout the notebook

from IPython.core.display import display, HTML
from jupyter_server.serverapp import list_running_servers


# Gets the proxied URL in the Kaggle Notebooks environment
def get_adk_proxy_url():
    PROXY_HOST = "https://kkb-production.jupyter-proxy.kaggle.net"
    ADK_PORT = "8000"

    servers = list(list_running_servers())
    if not servers:
        raise Exception("No running Jupyter servers found.")

    baseURL = servers[0]["base_url"]

    try:
        path_parts = baseURL.split("/")
        kernel = path_parts[2]
        token = path_parts[3]
    except IndexError:
        raise Exception(f"Could not parse kernel/token from base URL: {baseURL}")

    url_prefix = f"/k/{kernel}/{token}/proxy/proxy/{ADK_PORT}"
    url = f"{PROXY_HOST}{url_prefix}"

    styled_html = f"""
    <div style="padding: 15px; border: 2px solid #f0ad4e; border-radius: 8px; background-color: #fef9f0; margin: 20px 0;">
        <div style="font-family: sans-serif; margin-bottom: 12px; color: #333; font-size: 1.1em;">
            <strong>⚠️ IMPORTANT: Action Required</strong>
        </div>
        <div style="font-family: sans-serif; margin-bottom: 15px; color: #333; line-height: 1.5;">
            The ADK web UI is <strong>not running yet</strong>. You must start it in the next cell.
            <ol style="margin-top: 10px; padding-left: 20px;">
                <li style="margin-bottom: 5px;"><strong>Run the next cell</strong> (the one with <code>!adk web ...</code>) to start the ADK web UI.</li>
                <li style="margin-bottom: 5px;">Wait for that cell to show it is "Running" (it will not "complete").</li>
                <li>Once it's running, <strong>return to this button</strong> and click it to open the UI.</li>
            </ol>
            <em style="font-size: 0.9em; color: #555;">(If you click the button before running the next cell, you will get a 500 error.)</em>
        </div>
        <a href='{url}' target='_blank' style="
            display: inline-block; background-color: #1a73e8; color: white; padding: 10px 20px;
            text-decoration: none; border-radius: 25px; font-family: sans-serif; font-weight: 500;
            box-shadow: 0 2px 5px rgba(0,0,0,0.2); transition: all 0.2s ease;">
            Open ADK Web UI (after running cell below) ↗
        </a>
    </div>
    """

    display(HTML(styled_html))

    return url_prefix


print("✅ Helper functions defined.")

✅ Helper functions defined.


In [5]:
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1, # Initial delay before first retry (in seconds)
    http_status_codes=[429, 500, 503, 504] # Retry on these HTTP errors
)

# 4. 🤖 Medical Information Assistance 

1. Basic health-related queries (non-diagnostic)

2. Medication guidelines and general safety information

3. It serves as a first layer of intelligent support for patients seeking quick medical information.

In [6]:
iMedic = Agent(
    name="iMedic",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    description="A simple agent that can answer general questions.",
    instruction="You are a helpful assistant. Use Google Search for current info or if unsure.",
    tools=[google_search],
)

print("✅ Root Agent defined.")


✅ Root Agent defined.


# 5. 🔨 Custom Tools for ADK

In [7]:
from datetime import datetime

# Simple in-memory database (You can replace this with a real DB)
appointments = []

def book_doctor_appointment(patient_name: str,
                            doctor_name: str,
                            date: str,
                            time: str,
                            symptoms: str = "") -> dict:
    """
    Book a doctor appointment with validation.
    
    Args:
        patient_name (str): Name of the patient.
        doctor_name (str): Name of the doctor.
        date (str): Appointment date in YYYY-MM-DD format.
        time (str): Appointment time in HH:MM (24hr) format.
        symptoms (str): Optional - patient's symptoms.

    Returns:
        dict: Confirmation or error message.
    """

    # Validate date
    try:
        appointment_date = datetime.strptime(date, "%Y-%m-%d").date()
    except ValueError:
        return {"status": "error", "message": "Invalid date format. Use YYYY-MM-DD"}

    # Validate time
    try:
        appointment_time = datetime.strptime(time, "%H:%M").time()
    except ValueError:
        return {"status": "error", "message": "Invalid time format. Use HH:MM (24hr)"}

    # Check if time slot is already booked
    for appt in appointments:
        if (appt["doctor_name"].lower() == doctor_name.lower() and
            appt["date"] == date and
            appt["time"] == time):
            return {
                "status": "error",
                "message": f"Doctor {doctor_name} is already booked at {time} on {date}."
            }

    # Save appointment
    appointment = {
        "patient_name": patient_name,
        "doctor_name": doctor_name,
        "date": date,
        "time": time,
        "symptoms": symptoms
    }

    appointments.append(appointment)

    return {
        "status": "success",
        "message": "Appointment booked successfully!",
        "appointment": appointment
    }


# Example usage:

print("sucessfull")
print(book_doctor_appointment("John Doe", "Dr. Rahim", "2025-01-10", "14:30", "Headache"))

sucessfull
{'status': 'success', 'message': 'Appointment booked successfully!', 'appointment': {'patient_name': 'John Doe', 'doctor_name': 'Dr. Rahim', 'date': '2025-01-10', 'time': '14:30', 'symptoms': 'Headache'}}


# 6. ⏰ Doctor Appointment Booking
The agent can schedule doctor appointments in real time by:

Checking doctor availability

Suggesting next available slots

Confirming bookings

Updating or canceling appointments

Sending reminders or follow-ups

It integrates easily with custom tools.

In [8]:
# booking agent with custom function tools
booking_agent = LlmAgent(
    name="booking_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""You are a smart booking agent assistant.

    For currency conversion requests:
    1. Use `book_doctor_appointment()` for booking
    3. Check the "status" field in each tool's response for errors
    4.If any tool returns status "error", explain the issue to the user clearly.
    5. give the result json formet
    6. after booking the appoinment give it json formet
    """,
    tools=[book_doctor_appointment],
)

print("✅ Doctor booking agent created with custom function tools")
print("🔧 Available tools:")

✅ Doctor booking agent created with custom function tools
🔧 Available tools:


In [9]:
# Test the booking agent
runner = InMemoryRunner(agent=booking_agent)
print("✅ Runner created.")

✅ Runner created.


In [10]:
#response = await runner.run_debug(" book a doctor")

In [11]:
#response = await runner.run_debug("How to add this info")

In [12]:
response = await runner.run_debug("Kaykubad Dr. saynoor 2025/10/24 9.00AM skinissues")


 ### Created new session: debug_session_id

User > Kaykubad Dr. saynoor 2025/10/24 9.00AM skinissues


In [13]:
import uuid
from datetime import datetime

# Fake medicine inventory (replace with database in real app)
medicine_inventory = {
    "Paracetamol 500mg": {"price": 5.0, "stock": 50},
    "Aspirin 300mg": {"price": 8.0, "stock": 20},
    "Amoxicillin 250mg": {"price": 15.0, "stock": 10},
    "Cetirizine 10mg": {"price": 3.0, "stock": 40},
}

orders = []


def book_medicine_online(customer_name: str,
                         medicine_name: str,
                         quantity: int,
                         address: str,
                         phone: str):
    """
    Book/order medicine online with stock validation and 5% service charge.
    """

    # Validate input
    if quantity <= 0:
        return {"status": "error", "message": "Quantity must be greater than zero."}

    if medicine_name not in medicine_inventory:
        return {"status": "error", "message": f"{medicine_name} not found in inventory."}

    med = medicine_inventory[medicine_name]

    # Stock check
    if quantity > med["stock"]:
        return {
            "status": "error",
            "message": f"Only {med['stock']} units of {medicine_name} available."
        }

    # Base price
    base_price = quantity * med["price"]

    # Service charge 5%
    service_charge = base_price * 0.05

    # Final total
    total_price = base_price + service_charge

    # Update stock
    medicine_inventory[medicine_name]["stock"] -= quantity

    # Order data
    order = {
        "order_id": str(uuid.uuid4()),
        "customer_name": customer_name,
        "medicine_name": medicine_name,
        "quantity": quantity,
        "base_price": base_price,
        "service_charge_5_percent": service_charge,
        "total_price": total_price,
        "address": address,
        "phone": phone,
        "order_time": datetime.now().isoformat()
    }

    orders.append(order)

    return {
        "status": "success",
        "message": "Medicine ordered successfully!",
        "order": order
    }



In [14]:
# ---------- TEST ----------
print(book_medicine_online(
    "John Doe",
    "Paracetamol 500mg",
    3,
    "Dhaka, Bangladesh",
    "017xxxxxxxx"
))

{'status': 'success', 'message': 'Medicine ordered successfully!', 'order': {'order_id': '804790e4-7883-47a6-969d-9be523094b3f', 'customer_name': 'John Doe', 'medicine_name': 'Paracetamol 500mg', 'quantity': 3, 'base_price': 15.0, 'service_charge_5_percent': 0.75, 'total_price': 15.75, 'address': 'Dhaka, Bangladesh', 'phone': '017xxxxxxxx', 'order_time': '2025-11-24T23:03:22.289590'}}


# 7. 📝 Online Medicine Order & Confirmation
Checking pharmacy inventory

Processing user medicine lists

Calculating total price with delivery/service charges

Confirming and placing the order

Tracking delivery status

In [15]:
# booking agent with custom function tools
medicbooking_agent = LlmAgent(
    name="medicbooking_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""You are a smart booking agent assistant.

    For medicine order conversion requests:
    1. Use `book_medicine_online()` for booking
    3. Check the "status" field in each tool's response for errors
    4.If any tool returns status "error", explain the issue to the user clearly.
    5. give the result json formet
    6. ask everything and accept the data
    6. after confirm the order give it json formet with its service charge result and suceessful message.
    """,
    tools=[book_medicine_online],
)
runner = InMemoryRunner(agent=medicbooking_agent)
print("✅ Runner created.")

✅ Runner created.


In [16]:
response = await runner.run_debug("how to order medicine")


 ### Created new session: debug_session_id

User > how to order medicine
medicbooking_agent > I can help you with that! What is your name, what medicine do you want to order, what quantity do you need, what is your address, and what is your phone number?


In [17]:
response = await runner.run_debug("turag Paracetamol 500mg 5pcs Dhaka 019888888")
                                            


 ### Continue session: debug_session_id

User > turag Paracetamol 500mg 5pcs Dhaka 019888888
medicbooking_agent > Could you please provide your name?


In [18]:
response = await runner.run_debug('confirm this')


 ### Continue session: debug_session_id

User > confirm this
medicbooking_agent > I cannot confirm the order without your name. Please provide your name.


# 8. 🛠 Update the Agent's instruction and toolset

In [19]:
medical_booking_order_agent = LlmAgent(
    name="medical_booking_order_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    # Updated instruction
    instruction="""You are a smart doctor booking and medicine order assistant. You must strictly follow these steps and use the available tools.

  For any doctor booking or medicine order conversion request:

   1. for doctor booking: Use the book_doctor_appointment() tool to book the doctor for patient.
   2. for medicine order : Use the book_medicine_online() tool to confirm the order.
   3. Error Check: After each tool call, you must check the "status" field in the response. If the status is "error", you must stop and clearly explain the issue to the user.
   4. Provide Detailed Breakdown: In your summary, you must:
       * State the final result in json formet.
       * Explain how the result was how it work, including:
           * The fee percentage and the fee amount 
    """,
    tools=[book_doctor_appointment,
           book_medicine_online,
        AgentTool(agent=iMedic),  # Using another agent as a tool!
    ],
)

In [20]:
runner = InMemoryRunner(agent=medical_booking_order_agent)
print("✅ Runner created.")

✅ Runner created.


In [21]:
response = await runner.run_debug("what is your agent name?")


 ### Created new session: debug_session_id

User > what is your agent name?
medical_booking_order_agent > I am the medical_booking_order_agent. I can help you book doctor appointments and order medicines online.


In [22]:
#!adk create medical_booking_order_agent --model gemini-2.5-flash-lite --api_key $GOOGLE_API_KEY

In [23]:
#url_prefix = get_adk_proxy_url()

In [24]:
#!adk web --url_prefix {url_prefix}

# 9. 👾 🗃 Information Analysis and Report Generator Agent

In [25]:
from google.genai import types
from google.adk import Agent

data_analyst = Agent(
   model='gemini-2.5-flash',
   name="data_analyst",
   description="""Agent that analyzes the details on user insurance policy and 
   medical necessity for a pre-authorization request and creates a report on
   the same.""",
   instruction=f"""
   As an **Information Analysis and Report Generator Agent**, your primary role 
   is to evaluate pre-authorization requests for medical treatments.

    **Here's the process you will follow:**

    1.  **Receive Information:** You will be provided with two sets of crucial 
    information:
        * **User's Insurance Coverage Details:** Information pertaining to the 
        user's insurance policy and its coverage for the requested treatment.
        * **User's Medical Records:** Relevant medical history and details 
        concerning the treatment for which pre-authorization is sought.

    2.  **Analyze and Decide:**
        * Thoroughly **review and analyze** both the insurance coverage details 
        and the user's medical records.
        * Based on this analysis, make a **clear decision** to either **"Pass" 
        or "Reject"** the pre-authorization request.
        * Formulate a **reason for the decision**, explicitly referencing 
        relevant information from the patient's medical records and the 
        insurance policy eligibility criteria.

    3.  **Generate Report Content:**
        * Create the **content for a pre-authorization decision report** that 
        will be compiled into a PDF file.
        * The report must include:
            * **Patient Details:** Essential identifying information about the
            patient.
            * **Treatment Details:** A description of the treatment for which 
            pre-authorization was requested.
            * **Pre-authorization Decision:** Clearly state "Pass" or "Reject."
            * **Reason for Decision:** Provide the specific justification for 
            the decision, as determined in the previous step.
        * **Reference:** Use the provided sample report content as a **structural
         and formatting reference** for generating this report.

    4.  **Upload Document:**
        * Once the PDF content is generated, use the `store_pdf` tool to **upload 
        the content as a PDF file** to the designated Cloud Storage Bucket.

    5.  **Confirm to User:**
        * After the proposal report document's PDF content has been successfully 
        created and uploaded to the Cloud Storage Bucket, send a confirmation 
        message to the user, stating that "The pre-authorization decision report 
        has been created and uploaded to the Cloud Storage Bucket." Provide the 
        path of the GCS report location to the user. Prepend 
        "https://storage.cloud.google.com/" to the path instead of "gs:". 
        *Also provide a brief summary of the decision to the user.
        
    6. Here is a sample report 
    SAMPLE_REPORT=
*****************************Sample Report Document Template***********
Medical Pre-Authorization Request Report
Date of Report: July 19, 2025

Pre-Authorization for Elective LASIK Surgery (Rejection)
Patient Name: Mark Johnson

Treatment Name: Bilateral LASIK Eye Surgery

Insurance Provider: OptiCare Health Plans

Summary of Medical Records:
Patient Mark Johnson seeks LASIK surgery for refractive error correction to 
reduce dependence on eyeglasses. Corrected visual acuity is 20/25 in both eyes. 
No medical contraindications to continued use of corrective lenses or severe 
uncorrectable refractive error are noted.

Summary of Insurance Coverage:
OptiCare Health Plans policy (Policy ID: LASIK-ELEC-MJ-2024) was reviewed. The
policy explicitly categorizes LASIK as an elective/cosmetic procedure. Coverage
is provided only in cases of severe refractive error (e.g., >7.5 diopters) or 
documented medical intolerance to traditional corrective lenses 
(glasses/contacts). Neither of these criteria is met.

Pre-Authorization Claim Decision: REJECTED
Reason for Rejection: The requested procedure (Bilateral LASIK Eye Surgery) is 
categorized as an elective cosmetic procedure under the patient's OptiCare 
Health Plans policy and does not meet the strict medical necessity criteria for
coverage.
    """,
   generate_content_config=types.GenerateContentConfig(temperature=0.2),
   
)

In [26]:
import google.cloud.aiplatform as aiplatform
from google import genai
from google.genai import types
from typing import Optional
from dotenv import load_dotenv
from pathlib import Path
import warnings
import os
import logging
import base64


def extract_treatment_name(user_query: str) -> str:
    """
    Extracts the treatment name from a user query using the Gemini model.

    Args:
        user_query: The user's input query.

    Returns:
        The extracted treatment name, or an empty string if not found.
    """
    # Initialize the Gemini model
    client = genai.Client(
      vertexai=True,
      project=GOOGLE_CLOUD_PROJECT,  
      location=GOOGLE_CLOUD_LOCATION,           
  )

    model = "gemini-2.5-flash"

    # Construct a prompt for the Gemini model to extract the treatment name
    si_text = f"""
    From the following user query, extract only the name of the medical treatment.
    If no specific treatment is mentioned, respond with "None".

    User Query: "{user_query}"

    Treatment Name:
    """

    contents = [
    types.Content(
      role="user",
      parts=[
        types.Part.from_text(text=user_query)
      ]
    ),
    ]

    generate_content_config = types.GenerateContentConfig(
    temperature = 1,
    top_p = 1,
    seed = 0,
    max_output_tokens = 65535,
    system_instruction=[types.Part.from_text(text=si_text)],
    thinking_config=types.ThinkingConfig(
      thinking_budget=-1,
    ),
    )

    generated_text = ""
    try:
        # Stream the content generation and concatenate chunks
        for chunk in client.models.generate_content_stream(
        model = model,
        contents = contents,
        config = generate_content_config,
        ):
            generated_text += chunk.text
    except Exception as e:
        print(f"An error occurred during content generation: {e}")
        return "Error generating response."

    return generated_text.strip()

def extract_policy_information(policy_file: str, treatment_name: str) -> str:
    """
    Extracts all information related to a specific treatment from insurance policy pdf
    using the Gemini model.

    Args:
        policy_file: The full text of the insurance policy.
        treatment_name: The name of the treatment to find information about.

    Returns:
        Relevant information about the treatment, or a message indicating it's not found.
    """
    client = genai.Client(
        vertexai=True,
        project=GOOGLE_CLOUD_PROJECT,
        location=GOOGLE_CLOUD_LOCATION,
    )

    
    # with open(policy_file_path, 'rb') as pdf_file:
    #     pdf_bytes = pdf_file.read()

    

    # Define the text part of the prompt
    prompt_text = types.Part.from_text(text=f"""
    You are an AI assistant specialized in analyzing insurance policy documents.
    Given the following insurance policy text, extract all details and clauses
    specifically related to the medical treatment named \"{treatment_name}"\.
    Include information about coverage, exclusions, limits, conditions,
    and any other relevant terms.

    If \"{treatment_name}"\ is not explicitly mentioned or no information
    is found regarding it, state \\\"No specific information found for {treatment_name}.
    
    Insurance policy text: {policy_file}""")

    model = "gemini-2.5-flash"
    contents = [
        types.Content(
            role="user",
            parts=[
                prompt_text
            ]
        ),
    ]

    generate_content_config = types.GenerateContentConfig(
        temperature = 1,
        top_p = 0.95,
        max_output_tokens = 65535,
        thinking_config=types.ThinkingConfig(
            thinking_budget=0,
        ),
    )

    summary_output = ""
    # Stream the content generation
    for chunk in client.models.generate_content_stream(
        model = model,
        contents = contents,
        config = generate_content_config,
    ):
        summary_output += chunk.text
    return summary_output


def extract_medical_details(medical_report_file: str, treatment_name: str) -> str:
    """
    Extracts all information related to a specific treatment from medical report pdf
    using the Gemini model

    Args:
        medical_report_file (str): The full text of the medical report.
        treatment_name: The name of the treatment to find information about.

    Returns: A summary of the relevant medical details
    """
    client = genai.Client(
        vertexai=True,
        project=GOOGLE_CLOUD_PROJECT,
        location=GOOGLE_CLOUD_LOCATION,
    )


    # Define the text part of the prompt
    prompt_text = types.Part.from_text(text="""You are an AI assistant specialized in analyzing medical reports. 
    Given the following medical report txt, extract and summarize all relevant medical details specifically 
    related to the treatment named \"{treatment_name}\". Include information about diagnosis, treatment plans, 
    medications, procedures, and patient outcomes. If \"{treatment_name}\" is not explicitly mentioned or no
    information is found regarding it, state \\\"No specific information found for {treatment_name}.
    
    Medical report text {medical_report_file}""")


    model = "gemini-2.5-flash"
    contents = [
        types.Content(
            role="user",
            parts=[
                prompt_text
            ]
        ),
    ]

    generate_content_config = types.GenerateContentConfig(
        temperature = 1,
        top_p = 0.95,
        max_output_tokens = 65535,
        thinking_config=types.ThinkingConfig(
            thinking_budget=0,
        ),
    )

    summary_output = ""
    # Stream the content generation
    for chunk in client.models.generate_content_stream(
        model = model,
        contents = contents,
        config = generate_content_config,
    ):
        summary_output += chunk.text
    return summary_output

In [27]:
from google import genai
from google.adk.agents import Agent
from google.genai import types

information_extractor= Agent(
   model='gemini-2.5-flash',
   name="information_extractor",
   description="""Agent that extracts the details on user insurance policy and
   medical necessity for a pre-authorization request from documents provided 
   by the user.""",
   instruction=  """
    You are a information extractor agent that helps with extracting 
    details on treatment name from user request. You also extract policy
    details and medical test details on that treatment from respective
    documents provided by the user. You will have below information:
    1) user request containing treatment name for which they are seeking
    pre-authorization.
    2) a medical report containing details on tests and diagnosis for that
    treatment
    3) a insurance policy document containing details on insurance coverage
    and eligibility for that treatment.
    First you take the user request and extract the treatment name for which
    user is seeking pre-authorization using extract_treatment_name. Give
    medical record document path and extracted treatment name to
    extract_medical_details tool and extract details on medical records. Give
    insurance policy document path and extracted treatment name to
    extract_policy_information tool and extract details on policy. Call
    extract_medical_details and extract_policy_information parallely if you
    have both the documents.
    """,
   generate_content_config=types.GenerateContentConfig(temperature=0.2),
   tools=[extract_treatment_name, extract_policy_information, extract_medical_details],
)

# 10. 🦿 Medical Pre-Authorization Support agent

In [28]:
from google.adk.agents import Agent
from google.genai import types
from google.adk.tools.agent_tool import AgentTool

root_agent = Agent(
   model='gemini-2.5-flash',
   name='root_agent',
   description="""As a medical pre-authorization agent, you process user 
   pre-auth request for a treatment.""",

   instruction= """
    As a medical pre-authorization agent, you process user pre-auth request for
    a treatment. 

    Here's a breakdown of your responsibilities:
    1. You will extract relevant insurance details and medical details from 
    respective documents, specifically for the treatment user is seeking 
    pre-authorization for.
    2. You will analyze the necessity of the treatment based on the extracted 
    information, then verify user eligibility and coverage under their insurance 
    plan.
    3. Finally, you will create a report detailing the decision to accept or 
    reject your pre-authorization request.

    Here's how you should operate:
    1. Start by greeting the user and asking how I can help, providing a quick 
    overview of what you do.
    2. If the request isn't clear, you will ask some questions to understand 
    user needs better.
    3. You will need treatment name and two documents from the user to process 
    the pre-authorization request: one for their medical records related to the 
    treatment, and one for their health insurance policy. If these are not 
    promptly provided, you will explicitly ask the user to provide them, 
    clarifying which document is which. 
    4. Extract the treatment name that user is seeking pre-authorisation for 
    from the user's request using corresponding subagent.
    5. Extract medical details and insurance policy details correspondidng to 
    the treatment name from the documents provided by the user using 
    corresponding subagent.
    6. Next you will require to analyse the extracted content of the documents
    and provide a report detailing your decision on pre authorisation request. 
    You will call the corresponding subagent for this task.
    7. Based on the user's intent, determine which sub-agent is best suited to 
    handle the request.

    Ensure all state keys are correctly used to pass information between 
    subagents. 

    ***************************************************************************
    - Invoke the information_extractor subagent to extract treatment name from 
    user's request, also to extract details on user's medical records and 
    insurance coverage for this treatment. The information_extractor subagent 
    MUST return a comprehensive medical and insurance policy data extracted for 
    the specified pre auth request.
    - Invoke the data_analyst subagent to analyse data provided by information_extractor
    subagent and create a report detailing your decision on user's pre-authorization
    request for the treatment. The data_analyst_agent MUST have decision on 
    passing or rejecting the pre-auth request, create a report and store it in 
    designated path. Invoke the data_analyst subagent only after information_extractor
    has completed all its tasks.
    **********************************************************************************
    **********************************************************************************
    """,

   generate_content_config=types.GenerateContentConfig(temperature=0.2),

   tools=[
        AgentTool(agent=information_extractor),
        AgentTool(agent=data_analyst)
    ],
)

In [29]:
runner = InMemoryRunner(agent=root_agent)
print("✅ Runner created.")

✅ Runner created.


In [30]:

response = await runner.run_debug('Visual Acuity') 


 ### Created new session: debug_session_id

User > Visual Acuity


root_agent > Hello! I'm your medical pre-authorization agent. I can help you process your pre-authorization request for a treatment by extracting details from your medical records and insurance policy, then analyzing them to provide a decision report.

I see you're requesting pre-authorization for "Visual Acuity". To proceed, please upload two documents:
1. Your medical records related to the Visual Acuity treatment.
2. Your health insurance policy document.

Please let me know which document is which when you upload them.


In [31]:
response = await runner.run_debug("""
    HEALTH INSURANCE POLICY CERTIFICATE
    Issued by: GreenLife Health Insurance Ltd.

    Policy Number: GLH-2025-BD-4421
    Issue Date: 12-January-2025
    Policy Period: 12-January-2025 to 11-January-2026
    Type of Plan: Individual Health Secure
    Sum Insured: ৳7,00,000 (Seven Lakh Taka Only)

    Primary Policyholder Information:
    • Name: Mr. Tanvir Ahmed
    • Date of Birth: 18-February-1990
    • Relationship: Self
    • Address: House 19, Road 5, Sector 10, Uttara, Dhaka-1230
    • Contact Number: +880-1712-556677
    • Email: tanvir.ahmed@example.com

    Premium Details:
    • Annual Premium: ৳12,500
    • VAT (15%): ৳1,875
    • Total Premium Paid: ৳14,375

    Declared Health Conditions at Policy Inception:
    • Mild gastritis
    • Occasional migraine headaches
    """)


 ### Continue session: debug_session_id

User > 
    HEALTH INSURANCE POLICY CERTIFICATE
    Issued by: GreenLife Health Insurance Ltd.

    Policy Number: GLH-2025-BD-4421
    Issue Date: 12-January-2025
    Policy Period: 12-January-2025 to 11-January-2026
    Type of Plan: Individual Health Secure
    Sum Insured: ৳7,00,000 (Seven Lakh Taka Only)

    Primary Policyholder Information:
    • Name: Mr. Tanvir Ahmed
    • Date of Birth: 18-February-1990
    • Relationship: Self
    • Address: House 19, Road 5, Sector 10, Uttara, Dhaka-1230
    • Contact Number: +880-1712-556677
    • Email: tanvir.ahmed@example.com

    Premium Details:
    • Annual Premium: ৳12,500
    • VAT (15%): ৳1,875
    • Total Premium Paid: ৳14,375

    Declared Health Conditions at Policy Inception:
    • Mild gastritis
    • Occasional migraine headaches
    


root_agent > Thank you for providing your health insurance policy.

Now, please upload your medical records related to the "Visual Acuity" treatment. Once I have both documents, I can proceed with your pre-authorization request.


# 11. 🤖 iMedic -> Medical agent Working

In [32]:
medical_agent = LlmAgent(
    name="medical_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    # Updated instruction
    instruction="""You are a smart doctor booking and medicine order assistant and also medical pre-authorization agent, you process user pre-auth request for
    a treatment. You must strictly follow these steps and use the available tools.

  For any doctor booking or medicine order conversion request:

   1. for doctor booking: Use the book_doctor_appointment() tool to book the doctor for patient or use medical_booking_order_agent.
   2. for medicine order : Use the book_medicine_online() tool or medical_booking_order_agent   confirm the order.
   3. for medical pre-authorization agent : use agent root_agent 
   4. Error Check: After each tool call, you must check the "status" field in the response. If the status is "error", you must stop and clearly explain the issue to the user.
   5. Provide Detailed Breakdown: In your summary, you must:
       * State the final result in json formet.
       * Explain how the result was how it work, including:
           * The fee percentage and the fee amount 
    """,
    tools=[AgentTool(agent=root_agent),
        AgentTool(agent=medical_booking_order_agent),  # Using another agent as a tool!
    ],
)

In [33]:
runner = InMemoryRunner(agent=medical_agent)
print("✅ Runner created.")
response = await runner.run_debug('What type of agent you use? and what type of work you can do') 

✅ Runner created.

 ### Created new session: debug_session_id

User > What type of agent you use? and what type of work you can do
medical_agent > I am a smart doctor booking and medicine order assistant and also a medical pre-authorization agent. I can help you with:

1. **Doctor Booking:** Booking appointments with doctors.
2. **Medicine Orders:** Ordering medicines online.
3. **Medical Pre-authorization:** Processing pre-authorization requests for treatments.

I use the following tools for these tasks:
* `book_doctor_appointment()` for doctor bookings.
* `book_medicine_online()` for medicine orders.
* `root_agent` for medical pre-authorization requests.

I will always check the status of each tool call and inform you if there are any errors. For pre-authorization requests, I will provide a detailed breakdown of the result in JSON format, including fee percentages and amounts.


In [34]:
response = await runner.run_debug("what is medical pre_authorization")


 ### Continue session: debug_session_id

User > what is medical pre_authorization
medical_agent > Medical pre-authorization, also known as prior authorization or precertification, is a process where your health insurance company must approve a recommended diagnostic test, procedure, or service *before* you receive it.

This is done to ensure that the medical services you receive are medically necessary and appropriate for your condition, and to help control healthcare costs.

If you have a treatment plan that requires pre-authorization, you'll typically need to contact your insurance provider with specific information from your doctor. I can help you with this process using the `root_agent` tool.


# 12. 🚀 Memory Management - Sessions & Memory

In [35]:
from typing import Any, Dict

from google.adk.agents import Agent, LlmAgent
from google.adk.apps.app import App, EventsCompactionConfig
from google.adk.models.google_llm import Gemini
from google.adk.sessions import DatabaseSessionService
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.adk.tools.tool_context import ToolContext
from google.genai import types

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [36]:
# Define helper functions that will be reused throughout the notebook
async def run_session(
    runner_instance: Runner,
    user_queries: list[str] | str = None,
    session_name: str = "default",
):
    print(f"\n ### Session: {session_name}")

    # Get app name from the Runner
    app_name = runner_instance.app_name

    # Attempt to create a new session or retrieve an existing one
    try:
        session = await session_service.create_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )
    except:
        session = await session_service.get_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )

    # Process queries if provided
    if user_queries:
        # Convert single query to list for uniform processing
        if type(user_queries) == str:
            user_queries = [user_queries]

        # Process each query in the list sequentially
        for query in user_queries:
            print(f"\nUser > {query}")

            # Convert the query string to the ADK Content format
            query = types.Content(role="user", parts=[types.Part(text=query)])

            # Stream the agent's response asynchronously
            async for event in runner_instance.run_async(
                user_id=USER_ID, session_id=session.id, new_message=query
            ):
                # Check if the event contains valid content
                if event.content and event.content.parts:
                    # Filter out empty or "None" responses before printing
                    if (
                        event.content.parts[0].text != "None"
                        and event.content.parts[0].text
                    ):
                        print(f"{MODEL_NAME} > ", event.content.parts[0].text)
    else:
        print("No queries!")


print("✅ Helper functions defined.")

✅ Helper functions defined.


In [37]:
APP_NAME = "iMedic"  # Application
USER_ID = "default"  # User
SESSION = "default"  # Session
MODEL_NAME = "medical_agent"


# InMemorySessionService stores conversations in RAM (temporary)
session_service = InMemorySessionService()

# Create the Runner
runner = Runner(agent=medical_agent, app_name=APP_NAME, session_service=session_service)

print("✅ Stateful agent initialized!")
print(f"   - Application: {APP_NAME}")
print(f"   - User: {USER_ID}")
print(f"   - Using: {session_service.__class__.__name__}")

✅ Stateful agent initialized!
   - Application: iMedic
   - User: default
   - Using: InMemorySessionService


In [38]:
await run_session(
    runner,
    [
        "Hi, I am Jon Doe! I want to book a doctor",
        "Hello! What is my name?",  # This time, the agent should remember!
    ],
    "session_one",
)


 ### Session: session_one

User > Hi, I am Jon Doe! I want to book a doctor
medical_agent >  Sure, Jon! What type of doctor are you looking to book, and do you have any preferred dates or times?

User > Hello! What is my name?
medical_agent >  Your name is Jon Doe.


In [39]:
await run_session(
    runner,
    [
        "what is I asking before",
    ],
    "session_one",
)


 ### Session: session_one

User > what is I asking before
medical_agent >  You previously asked to book a doctor.


# **12.1 🔎 Persistent Sessions with DatabaseSessionService**

In [40]:
# SQLite database will be created automatically
db_url = "sqlite:///my_agent_data.db"  # Local SQLite file
session_service = DatabaseSessionService(db_url=db_url)

# Step 3: Create a new runner with persistent storage
runner = Runner(agent=medical_agent, app_name=APP_NAME, session_service=session_service)

print("✅ Upgraded to persistent sessions!")
print(f"   - Database: my_agent_data.db")
print(f"   - Sessions will survive restarts!")

✅ Upgraded to persistent sessions!
   - Database: my_agent_data.db
   - Sessions will survive restarts!


In [41]:
await run_session(
    runner,
    [
        "Hi, I am Jon Doe! I want to book a doctor",
        "Hello! What is my name?",  # This time, the agent should remember!
    ],
    "session_one",
)


 ### Session: session_one

User > Hi, I am Jon Doe! I want to book a doctor
medical_agent >  Great, Jon! What type of doctor are you looking to book, and do you have any preferred times or locations?

User > Hello! What is my name?
medical_agent >  You are Jon Doe.



In [42]:
import sqlite3

def check_data_in_db():
    with sqlite3.connect("my_agent_data.db") as connection:
        cursor = connection.cursor()
        result = cursor.execute(
            "select app_name, session_id, author, content from events"
        )
        print([_[0] for _ in result.description])
        for each in result.fetchall():
            print(each)


check_data_in_db()

['app_name', 'session_id', 'author', 'content']
('iMedic', 'session_one', 'user', '{"parts": [{"text": "Hi, I am Jon Doe! I want to book a doctor"}], "role": "user"}')
('iMedic', 'session_one', 'medical_agent', '{"parts": [{"text": "Great, Jon! What type of doctor are you looking to book, and do you have any preferred times or locations?"}], "role": "model"}')
('iMedic', 'session_one', 'user', '{"parts": [{"text": "Hello! What is my name?"}], "role": "user"}')
('iMedic', 'session_one', 'medical_agent', '{"parts": [{"text": "You are Jon Doe.\\n"}], "role": "model"}')


# 12.2 🧠 Memory Management

In [43]:
from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.memory import InMemoryMemoryService
from google.adk.tools import load_memory, preload_memory
from google.genai import types

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [44]:
async def run_session(
    runner_instance: Runner, user_queries: list[str] | str, session_id: str = "default"
):
    """Helper function to run queries in a session and display responses."""
    print(f"\n### Session: {session_id}")

    # Create or retrieve session
    try:
        session = await session_service.create_session(
            app_name=APP_NAME, user_id=USER_ID, session_id=session_id
        )
    except:
        session = await session_service.get_session(
            app_name=APP_NAME, user_id=USER_ID, session_id=session_id
        )

    # Convert single query to list
    if isinstance(user_queries, str):
        user_queries = [user_queries]

    # Process each query
    for query in user_queries:
        print(f"\nUser > {query}")
        query_content = types.Content(role="user", parts=[types.Part(text=query)])

        # Stream agent response
        async for event in runner_instance.run_async(
            user_id=USER_ID, session_id=session.id, new_message=query_content
        ):
            if event.is_final_response() and event.content and event.content.parts:
                text = event.content.parts[0].text
                if text and text != "None":
                    print(f"Model: > {text}")


print("✅ Helper functions defined.")

✅ Helper functions defined.


In [45]:
memory_service = (
    InMemoryMemoryService()
)  # ADK's built-in Memory Service for development and testing

In [46]:
async def auto_save_to_memory(callback_context):
    """Automatically save session to memory after each agent turn."""
    await callback_context._invocation_context.memory_service.add_session_to_memory(
        callback_context._invocation_context.session
    )


print("✅ Callback created.")

✅ Callback created.


In [47]:
medical_agent = LlmAgent(
    name="medical_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    # Updated instruction
    instruction="""You are a smart doctor booking and medicine order assistant and also medical pre-authorization agent, you process user pre-auth request for
    a treatment. You must strictly follow these steps and use the available tools.

  For any doctor booking or medicine order conversion request:

   1. for doctor booking: Use the book_doctor_appointment() tool to book the doctor for patient or use medical_booking_order_agent.
   2. for medicine order : Use the book_medicine_online() tool or medical_booking_order_agent   confirm the order.
   3. for medical pre-authorization agent : use agent root_agent 
   4. Error Check: After each tool call, you must check the "status" field in the response. If the status is "error", you must stop and clearly explain the issue to the user.
   5. Provide Detailed Breakdown: In your summary, you must:
       * State the final result in json formet.
       * Explain how the result was how it work, including:
           * The fee percentage and the fee amount 
    """,
    tools=[AgentTool(agent=root_agent),
        AgentTool(agent=medical_booking_order_agent),
           preload_memory, # Using another agent as a tool!
    ],
    after_agent_callback=auto_save_to_memory,  # Saves after each turn!
)

In [48]:
# Create a runner for the auto-save agent
# This connects our automated agent to the session and memory services
auto_runner = Runner(
    agent=medical_agent,  # Use the agent with callback + preload_memory
    app_name=APP_NAME,
    session_service=session_service,  # Same services from
    memory_service=memory_service,
)

print("✅ Runner created.")

✅ Runner created.


In [49]:
await run_session(
    auto_runner,
    "What is your agent name",
    "auto-save-test"
)


### Session: auto-save-test

User > What is your agent name
Model: > I am the `medical_agent`.


In [50]:
# The callback will automatically save this to memory when the turn completes
await run_session(
    auto_runner,
    " My name is Jon",
    "auto-save-test",
)


### Session: auto-save-test

User >  My name is Jon
Model: > Hi Jon, how can I help you today?



In [51]:
await run_session(
    auto_runner,
    " without those what can help you do?",
    "auto-save-test-2",
)


### Session: auto-save-test-2

User >  without those what can help you do?
Model: > I can help you with the following:
1. **Book a doctor's appointment**: I can find and book appointments with doctors for you.
2. **Order medicines online**: I can help you order your prescribed medicines.
3. **Process medical pre-authorization requests**: I can assist with pre-authorization for treatments.


# 13. 🔎 Agent Observability + Agent Evaluation

In [52]:
import logging
import os

# Clean up any previous logs
for log_file in ["logger.log", "web.log", "tunnel.log"]:
    if os.path.exists(log_file):
        os.remove(log_file)
        print(f"🧹 Cleaned up {log_file}")

# Configure logging with DEBUG log level.
logging.basicConfig(
    filename="logger.log",
    level=logging.DEBUG,
    format="%(filename)s:%(lineno)s %(levelname)s:%(message)s",
)

print("✅ Logging configured")

✅ Logging configured


# 📚 Learn More 
**1. [github](https://github.com/turagsarkar/5-Day-AI-Agents-Intensive-Course-with-Google)**
**2. [google adk](http://https://google.github.io/adk-docs/)**


Author
[turagsarkar](http://https://github.com/turagsarkar)